In [9]:
import os
import json
import copy
import numpy as np
import pandas as pd

In [10]:
rewards_per_ep = pd.read_csv('data/reward_log.txt',header=None)

In [11]:
rewards_per_ep = rewards_per_ep[0].apply(lambda x: int(x.split()[1]))

In [12]:
dfs = []
for exp_f in os.listdir('data'):
    if not exp_f.startswith('expert') or not exp_f.endswith('.txt'):
        continue

    print('reading',exp_f)
    f = open(f'data/{exp_f}')
    data = json.load(f)
    f.close()

    df = []
    for k, v in data.items():
        o, a = v
        o.append(a)
        df.append(o)

    df = pd.DataFrame(df).iloc[:,:129]
    dfs.append(df)

reading expert_2022-04-18T15-08-29.txt
reading expert_2022-04-18T15-22-57.txt
reading expert_2022-04-18T15-26-55.txt
reading expert_2022-04-18T15-31-12.txt
reading expert_2022-04-18T15-34-37.txt
reading expert_2022-04-18T15-37-46.txt
reading expert_2022-04-18T15-40-20.txt
reading expert_2022-04-18T15-43-02.txt
reading expert_2022-04-18T15-46-45.txt
reading expert_2022-04-18T15-49-47.txt
reading expert_2022-04-18T17-45-40.txt
reading expert_2022-04-18T17-52-24.txt
reading expert_2022-04-18T17-54-38.txt
reading expert_2022-04-18T17-58-00.txt
reading expert_2022-04-18T18-00-32.txt
reading expert_2022-04-18T18-03-22.txt
reading expert_2022-04-18T18-05-46.txt
reading expert_2022-04-18T18-08-26.txt
reading expert_2022-04-18T18-11-25.txt
reading expert_2022-04-18T18-14-07.txt


In [13]:
pd.concat(dfs).isna().sum().value_counts()

0    129
dtype: int64

In [14]:
train = pd.concat(dfs)
train.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,128
0,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
1,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
2,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
3,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3
4,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,2,66,240,146,215,3


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23484 entries, 0 to 1177
Columns: 129 entries, 0 to 128
dtypes: int64(129)
memory usage: 23.3 MB


In [16]:
X = train.drop(128, axis=1)
y = train[128]

In [17]:
y = pd.get_dummies(y)
y.insert(0, 0, 0)
y.insert(5, 5, 0)
y.insert(6, 6, 0)
y.insert(7, 7, 0)
y.insert(8, 8, 0)

In [18]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
1,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
2,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
3,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215
4,0,112,114,115,0,3,88,88,88,88,...,0,0,0,0,0,2,66,240,146,215


In [19]:
y.head()

,0,1,2,3,4,5,6,7,8
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0


In [20]:
X.shape

(23484, 128)

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim

In [22]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [25]:
class DDQN(nn.Module):
    """
    The Double Deep Q-Network has as input a state s and
    outputs the state-action values q(s,a_1), ..., q(s,a_n) for all n actions.
    :param: state_dim: for input layer
    :param: hidden_dim: for every hidden layer
    :param: action_dim: for output layer
    """
    def __init__(self, action_dim, state_dim, hidden_dim):
        super().__init__()

        self.online = nn.Sequential(
            nn.Linear(state_dim, hidden_dim*2),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.LeakyReLU(),
            nn.Linear(hidden_dim, action_dim)
        )

        self.target = copy.deepcopy(self.online)

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        """
        When doing update by forward, it takes:
        :param: input: all state of each observation
        :param: model: online or target
        :return: Q_values of all actions given state from online/target
        """

        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [18]:
def train(model, epochs, X_train, y_train, batch_size=32):

    model = model
    
    loss_fn = nn.SmoothL1Loss()
    optimizer = optim.Adam(net.parameters(), lr=0.0001)
    
    train_loss = [loss_fn(model(X_train, 'online'), y_train).item()]
    print(f'epoch: {0}, train loss = {train_loss[0]: .4f}')
    
    running_loss = 0.0
    for epoch in range(epochs):
        
        # random the order of X_train and y_train for batching
        permutation = torch.randperm(X_train.size()[0])
        
        avg_train_loss = []
        
        for i in range(0, X_train.size()[0], batch_size):
            
            indices = permutation[i:i+batch_size]
            batch_x, batch_y = X_train[indices], y_train[indices]
            
            # init optimizer
            optimizer.zero_grad()

            # forward pass and loss
            batch_y_hat = model(batch_x, 'online')
            loss = loss_fn(batch_y_hat, batch_y)

            # backward pass
            loss.backward()

            # update
            optimizer.step()
            
            avg_train_loss.append(loss.item())
        
        train_loss.append(np.mean(avg_train_loss))

        if (epoch + 1) % 10 == 0:
            print(f'epoch: {epoch+1}, train loss = {loss.item(): .4f}')
    
    return train_loss

In [23]:
X = torch.tensor(X.values, dtype=torch.float32).to(device)
y = torch.tensor(y.values, dtype=torch.float32).to(device)

In [20]:
net = DDQN(9,128,128).to(device)

In [21]:
train(net, 1000, X, y)

epoch: 0, train loss =  8.6661
epoch: 10, train loss =  0.0319
epoch: 20, train loss =  0.0440
epoch: 30, train loss =  0.0311
epoch: 40, train loss =  0.0220
epoch: 50, train loss =  0.0190
epoch: 60, train loss =  0.0335
epoch: 70, train loss =  0.0305
epoch: 80, train loss =  0.0178
epoch: 90, train loss =  0.0214
epoch: 100, train loss =  0.0257
epoch: 110, train loss =  0.0192
epoch: 120, train loss =  0.0164
epoch: 130, train loss =  0.0170
epoch: 140, train loss =  0.0186
epoch: 150, train loss =  0.0222
epoch: 160, train loss =  0.0084


KeyboardInterrupt: 

In [22]:
net.target.load_state_dict(net.online.state_dict())

<All keys matched successfully>

In [23]:
torch.save(dict(model=net.state_dict(), exploration_rate=1.0), "data/expert4.chkpt")

In [33]:
model = DDQN(9,128,128).to(device)
loss_fn = nn.SmoothL1Loss()
checkpoint = torch.load("data/expert4.chkpt")
model.load_state_dict(checkpoint['model'])
y_pred = model(X, 'online')
loss_fn(y_pred, y).item()

0.013653799891471863